# WonderCat Notebook

Brian's Daley Github: https://github.com/uconndxlab/wondercat/wiki/Date-Query-Reference

In [1]:
# Import necessary libraries.
import re, warnings, sys, os, base64, requests, json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import networkx as nx

from functions import *
import constants

# # Declare directory.
# abs_dir = Path(__file__).parent
# data = pd.read_csv(abs_dir / 'Data/reading-experiences.csv', sep = ',')

# Call Data from WordPress API
wp_call = read_wordpress_post_with_pagination()

# Reshape wp_call (json) as dataframe.
data = transform_to_dataframe(wp_call)

data.head()

,id,author,date,benefit,experience,technology,title,QID
0,710,1,2025-01-24T19:18:33,xxx-I need to enter something new,xxx-I need to enter something new,xxx-I need to enter something new,testing,Q3456789
1,474,4,2025-01-15T19:26:15,Bias Reduction,Alienation,Almighty Heart + Soliloquy,Test,Q223880
2,362,5,2025-01-09T21:27:04,Faith,Wonder,Enigma,Mystery Plays,Q240911
3,364,5,2025-01-09T20:56:26,Generosity,Wonder,Stretch,Oedipus,Q148643
4,363,5,2025-01-09T20:55:14,Faith,Wonder,Plot Twist,Oedipus,Q148643


## Networks

In [2]:
%%time

G = create_network_graph(data, 'author', 'benefit', 'experience', 'title')

CPU times: user 317 ms, sys: 21.5 ms, total: 339 ms
Wall time: 412 ms


In [3]:
%%time

# Create network visualization with Plotly.
def create_network_visualization(graph_object):
    node_traces = create_node_traces(graph_object)
    edge_traces = create_edge_traces(graph_object)

    fig = px.line(
        edge_traces, x = 'x', y = 'y'
    ).update_traces(line_color='#888')

    node_trace = px.scatter(
        node_traces, x="x", y="y", 
        color="type", size='size', opacity = 1,
        hover_data=['text']
    ).data

    fig.add_traces(node_trace)

    return fig

create_network_visualization(G)

CPU times: user 752 ms, sys: 564 ms, total: 1.32 s
Wall time: 5.23 s


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'x=%{x}<br>y=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': '#888', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array([ 0.00013233, -0.01361577,         nan, ..., -0.07616096, -0.08209036,
                                  nan], shape=(969,)),
              'xaxis': 'x',
              'y': array([-0.00081388, -0.09377485,         nan, ...,  0.14185539,  0.24678261,
                                  nan], shape=(969,)),
              'yaxis': 'y'},
             {'customdata': array([[5],
                                   [1],
                                   [4]], dtype=object),
              'hovertemplate': ('type=author<br>x=%{x}<br>y=%{y' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'author',
              'marker': {'color': '#636efa',
                         'opacity': 1,
                         'size': array([64, 14,  2]),
                         'sizemode': 'area',
                         'sizeref': np.float64(0.16),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'author',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([ 0.00013233, -0.04932962, -0.00438065]),
              'xaxis': 'x',
              'y': array([-0.00081388,  0.01563648, -0.15331912]),
              'yaxis': 'y'},
             {'customdata': array([['Peace of Mind'],
                                   ['Intimacy'],
                                   ['Innovative Thinking'],
                                   ['Self-Scrutiny'],
                                   ['Forgiveness'],
                                   ['Purpose'],
                                   ['Vigilance'],
                                   ['Unrealistic Romantic Expectations'],
                                   ['Expanded Inner List of Cultural Norms'],
                                   ['Creative Breakthrough'],
                                   ['Tend and Befriend'],
                                   ['Creativity'],
                                   ['Healthy Grief'],
                                   ['Realistic Appreciation of Love'],
                                   ["Humility about One's Ideas"],
                                   ['Bias Reduction'],
                                   ['Obedience'],
                                   ['Community'],
                                   ['Anxiety'],
                                   ['Healing'],
                                   ['Cognitive Flexibility'],
                                   ['Belief'],
                                   ['Gratitude'],
                                   ['Gratitude without Inferiority'],
                                   ['Catastrophizing'],
                                   ['Faith'],
                                   ['Detached Self-Awareness'],
                                   ['Generosity'],
                                   ["Humility about One's Nature"]], dtype=object),
              'hovertemplate': ('type=benefit<br>x=%{x}<br>y=%{' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'benefit',
              'marker': {'color': '#EF553B',
                         'opacity': 1,
                         'size': array([10,  8,  4,  6,  4,  4,  8,  4,  4,  4,  6,  4,  4,  6,  4,  8,  4,  8,
                                         4,  8,  6,  6,  6,  4,  4,  4,  4,  4,  4]),
                         'sizemode': 'area',
                         'sizeref': np.float64(0.16),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'benefit',
              'orientation': 'v',
              'sh

## Call WordPress API

In [7]:
%%time

# Import credentials
WP_USER = constants.WP_USER
WP_KEY = constants.WP_KEY
wp_credentials = WP_USER + WP_KEY
wp_credentials = WP_USER + WP_KEY
wp_token = base64.b64encode(wp_credentials.encode())
wp_header = {'Authorization': 'Basic ' + wp_token.decode('utf-8')}

# response = requests.get('https://env-1120817.us.reclaim.cloud/wp-json/wp/v2/experience?page=1&per_page=100', headers = wp_header)
# print (response)
# print (response.json())

def read_wordpress_posts():
    api_url = 'https://env-1120817.us.reclaim.cloud/wp-json/wp/v2/user-experience'
    response = requests.get(api_url)
    response_json = response.json()
    print(response_json)

def get_total_pagecount():
    api_url = 'https://env-1120817.us.reclaim.cloud/wp-json/wp/v2/user-experience?page=1&per_page=100'
    response = requests.get(api_url)
    pages_count = response.headers['X-WP-TotalPages']
    return int(pages_count)

def read_wordpress_post_with_pagination():
    total_pages = get_total_pagecount()
    current_page = 1
    all_page_items_json = []
    while current_page <= total_pages:
        api_url = f"https://env-1120817.us.reclaim.cloud/wp-json/wp/v2/user-experience?page={current_page}&per_page=100"
        page_items = requests.get(api_url)
        page_items_json = page_items.json()
        all_page_items_json.extend(page_items_json)
        current_page = current_page + 1
    return all_page_items_json

wp_data = read_wordpress_post_with_pagination()
wp_data[0]

CPU times: user 312 ms, sys: 90.3 ms, total: 402 ms
Wall time: 5.19 s


{'id': 474,
 'date': '2025-01-15T19:26:15',
 'date_gmt': '2025-01-15T19:26:15',
 'guid': {'rendered': 'https://env-1120817.us.reclaim.cloud/?post_type=experience&#038;p=474'},
 'modified': '2025-01-15T19:42:40',
 'modified_gmt': '2025-01-15T19:42:40',
 'slug': 'brian-test',
 'status': 'publish',
 'type': 'user-experience',
 'link': 'https://env-1120817.us.reclaim.cloud/user-experience/brian-test/',
 'title': {'rendered': 'Brian Test'},
 'content': {'rendered': '', 'protected': False},
 'author': 4,
 'featured_media': 458,
 'template': '',
 'meta': {'_acf_changed': True},
 'benefit': [{'id': 179,
   'name': 'Bias Reduction',
   'slug': 'bias-reduction',
   'description': ''}],
 'experience': [{'id': 83,
   'name': 'Alienation',
   'slug': 'alienation',
   'description': 'Distrusting yourself. Alientation is the inverse of paranoia, which involves distrusting someone or something outside of yourself.\r\n\r\nAlso called: Distrust\r\n\r\n'}],
 'technology': [{'id': 114,
   'name': 'Almight

### Convert Request JSON to DataFrame
Columsn to keep: source_narratives, benefit, experience, 

In [188]:
for k,v in wp_data[0].items():
    print (k)

id
count
description
link
name
slug
taxonomy
meta
acf
_links


In [8]:
%%time

wp_data[3]['benefit'][0]['name']

CPU times: user 7 μs, sys: 1 μs, total: 8 μs
Wall time: 11.7 μs


'Faith'

In [9]:
%%time

def extract_values(dictionary):
    name = dictionary['name']
    # name = dictionary.get('name', None)
    return name

wp_test = pd.DataFrame(wp_data)
wp_test = wp_test[['id', 'author', 'date', 'benefit', 'experience', 'technology', 'acf']]
wp_test['dumb'] = pd.json_normalize(wp_test['benefit'])
wp_test['benefit'] = pd.json_normalize(wp_test['dumb'])['name']


# wp_test['benefit'] = pd.DataFrame.from_dict(wp_test)
# wp_test['benefit'] = wp_test['benefit'].apply(lambda x: pd.Series(extract_values(x)))

# for k,v in test['benefit'].items():
#     print (k)

# [n['name'] for n in test['benefit']]

# for r in test['benefit']:
#     print (r['name'])

# benefits = test['benefit']

# wp_test['new'] = pd.json_normalize(wp_test['benefit'])['name']
wp_test.head()

CPU times: user 13.6 ms, sys: 3.37 ms, total: 17 ms
Wall time: 34.9 ms


,id,author,date,benefit,experience,technology,acf,dumb
0,474,4,2025-01-15T19:26:15,Bias Reduction,"[{'id': 83, 'name': 'Alienation', 'slug': 'ali...","[{'id': 114, 'name': 'Almighty Heart + Soliloq...","{'title_of_creative_work': 'Test', 'in_wikidat...","{'id': 179, 'name': 'Bias Reduction', 'slug': ..."
1,362,5,2025-01-09T21:27:04,Faith,"[{'id': 29, 'name': 'Wonder', 'slug': 'wonder'...","[{'id': 199, 'name': 'Enigma', 'slug': 'enigma...","{'title_of_creative_work': 'Mystery Plays', 'i...","{'id': 69, 'name': 'Faith', 'slug': 'faith', '..."
2,364,5,2025-01-09T20:56:26,Generosity,"[{'id': 29, 'name': 'Wonder', 'slug': 'wonder'...","[{'id': 170, 'name': 'Stretch', 'slug': 'stret...","{'title_of_creative_work': 'Oedipus', 'in_wiki...","{'id': 61, 'name': 'Generosity', 'slug': 'gene..."
3,363,5,2025-01-09T20:55:14,Faith,"[{'id': 29, 'name': 'Wonder', 'slug': 'wonder'...","[{'id': 150, 'name': 'Plot Twist', 'slug': 'pl...","{'title_of_creative_work': 'Oedipus', 'in_wiki...","{'id': 69, 'name': 'Faith', 'slug': 'faith', '..."
4,361,5,2025-01-09T20:51:11,Peace of Mind,"[{'id': 111, 'name': 'Tranquility', 'slug': 't...","[{'id': 169, 'name': 'Stream of Consciousness'...","{'title_of_creative_work': 'Me Before You', 'i...","{'id': 192, 'name': 'Peace of Mind', 'slug': '..."


In [29]:
%%time

def extract_values(dictionary):
    name = dictionary['name']
    return name

def transform_to_dataframe(api_call):
    api_data = pd.DataFrame(api_call)
    api_data = api_data[['id', 'author', 'date', 'benefit', 'experience', 'technology', 'acf']] # Select columns to work with. Add 'wikidata' when ready.
    api_data['title'] = pd.json_normalize(api_data['acf'])['title_of_creative_work']
    api_data['QID'] = pd.json_normalize(api_data['acf'])['wikidata-qid']
    # This should be cleaner...
    api_data['bene_del'] = pd.json_normalize(api_data['benefit'])
    api_data['benefit'] = pd.json_normalize(api_data['bene_del'])['name']
    api_data['exp_del'] = pd.json_normalize(api_data['experience'])
    api_data['experience'] = pd.json_normalize(api_data['exp_del'])['name']
    api_data['tech_del'] = pd.json_normalize(api_data['technology'])
    api_data['technology'] = pd.json_normalize(api_data['tech_del'])['name']
    del api_data['acf'], api_data['bene_del'], api_data['exp_del'], api_data['tech_del']

    return api_data

data = transform_to_dataframe(wp_data)

data.head()

CPU times: user 6.4 s, sys: 437 ms, total: 6.84 s
Wall time: 7.81 s


,id,author,date,benefit,experience,technology,title,QID
0,474,4,2025-01-15T19:26:15,Bias Reduction,Alienation,Almighty Heart + Soliloquy,Test,Q223880
1,362,5,2025-01-09T21:27:04,Faith,Wonder,Enigma,Mystery Plays,Q240911
2,364,5,2025-01-09T20:56:26,Generosity,Wonder,Stretch,Oedipus,Q148643
3,363,5,2025-01-09T20:55:14,Faith,Wonder,Plot Twist,Oedipus,Q148643
4,361,5,2025-01-09T20:51:11,Peace of Mind,Tranquility,Stream of Consciousness,Me Before You,Q20657314


## Network

In [92]:
%%time

# Create network graph from dataframe (df) and three columns (c1, c2, c3),
# where c1 is the source of c2; c2 is the source of c3.
def create_network_graph(df, c1, c2, c3):
    c1_c2 = df[[c1, c2]].rename(columns = {c1:'source', c2:'target'})
    c2_c3 = df[[c2, c3]].rename(columns = {c2:'source', c3:'target'})
    # c3_c4 = df[[c3, c4]].rename(columns = {c3:'source', c4:'target'})

    df_graph = pd.concat([c1_c2, c2_c3], ignore_index = True) # c3_c4
    df_graph = df_graph.query('source != "source"') # Ensure a "source" not isn't present.

    # Create weight column based on occurences of source-target pairings.
    df_graph = pd.DataFrame(df_graph.value_counts()).reset_index()
    df_graph = df_graph.rename(columns={'count':'weight'})

    # Initialize graph object.
    G = nx.from_pandas_edgelist(df_graph, 'source', 'target', 'weight')

    # Remove self-loops.
    G.remove_edges_from(nx.selfloop_edges(G))

    # Add nodes.
    nodes = list(dict.fromkeys(df_graph['source'].values.tolist() + df_graph['target'].values.tolist()))
    nodes = pd.DataFrame(nodes, columns = ['source'])
    G.add_nodes_from(nodes)

    # Set degree attributes.
    nx.set_node_attributes(G, dict(G.degree(G.nodes())), 'degree')

    # Set node type attributes.
    node_attrs = df.to_dict('list')
    node_attrs = {k: node_attrs[k] for k in (c1, c2, c3)}
    node_attrs = {v: k for k, values in node_attrs.items() for v in values}
    print (node_attrs)
    nx.set_node_attributes(G, node_attrs, 'type')

    # # Find centrality measures.
    # betweenness_dict = nx.betweenness_centrality(G)
    # eigenvector_dict = nx.eigenvector_centrality(G)
    # degree_cent_dict = nx.degree_centrality(G)

    # # Assign each centrality measure to an attribute.
    # nx.set_node_attributes(G, betweenness_dict, 'betweenness')
    # nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')
    # nx.set_node_attributes(G, degree_cent_dict, 'degree_cent')

    # Declare node and edge positions with seed for reproducibility.
    pos = nx.spring_layout(G, seed=7)
    nx.set_node_attributes(G, pos, 'pos')

    # # Find communities.
    # communities = nx.community.naive_greedy_modularity_communities(G)

    # # Create a dictionary that maps nodes to their community.
    # modularity_dict = {}
    # for i, c in enumerate(communities):
    #     for name in c:
    #         modularity_dict[name] = i
    
    # nx.set_node_attributes(G, modularity_dict, 'group')

    # Double check "source" node is removed.
    G.remove_node('source')

    return (G)

G = create_network_graph(data, 'title', 'experience', 'benefit')
print (G)

{'Test': 'title', 'Mystery Plays': 'title', 'Oedipus': 'title', 'Me Before You': 'title', 'The Crying of Lot 49': 'title', 'Saturday': 'title', 'The Intuitionist': 'title', 'Mrs. Dalloway': 'title', 'In Search of Lost Time': 'title', 'Pilgrimage': 'title', 'Middlemarch': 'title', 'Turn of the Screw': 'title', 'The Black Cat': 'title', 'Buffy the Vampire Slayer': 'title', 'Mary Poppins': 'title', 'Horse Feathers': 'title', 'The Sandman': 'title', 'Broad City': 'title', 'Crazy Ex-Girlfriend': 'title', 'Miranda': 'title', '30 Rock': 'title', 'Don Quixote': 'title', 'Blazing Saddles': 'title', 'A Funny Thing Happened on the Way to the Forum': 'title', 'I Love Lucy': 'title', 'Lysistrata': 'title', 'The Frogs': 'title', "Gulliver's Travels": 'title', 'Bioshock': 'title', 'Persepolis 2: The Story of a Return': 'title', 'The Great Railway Bazaar: By Train Through Asia': 'title', 'The Left Hand of Darkness': 'title', 'Utopia': 'title', 'An Apology for the Life of Mrs. Shamela Andrews': 'title'

## Plotly Network Visualization

To do:
* Associate nodes with metadata (user, experience, etc. for shape)
* Change color scheme of network to reflect communities

In [96]:
%%time

G = create_network_graph(data, 'title', 'experience', 'benefit')
G
# create_network_visualization(G)

{'Test': 'title', 'Mystery Plays': 'title', 'Oedipus': 'title', 'Me Before You': 'title', 'The Crying of Lot 49': 'title', 'Saturday': 'title', 'The Intuitionist': 'title', 'Mrs. Dalloway': 'title', 'In Search of Lost Time': 'title', 'Pilgrimage': 'title', 'Middlemarch': 'title', 'Turn of the Screw': 'title', 'The Black Cat': 'title', 'Buffy the Vampire Slayer': 'title', 'Mary Poppins': 'title', 'Horse Feathers': 'title', 'The Sandman': 'title', 'Broad City': 'title', 'Crazy Ex-Girlfriend': 'title', 'Miranda': 'title', '30 Rock': 'title', 'Don Quixote': 'title', 'Blazing Saddles': 'title', 'A Funny Thing Happened on the Way to the Forum': 'title', 'I Love Lucy': 'title', 'Lysistrata': 'title', 'The Frogs': 'title', "Gulliver's Travels": 'title', 'Bioshock': 'title', 'Persepolis 2: The Story of a Return': 'title', 'The Great Railway Bazaar: By Train Through Asia': 'title', 'The Left Hand of Darkness': 'title', 'Utopia': 'title', 'An Apology for the Life of Mrs. Shamela Andrews': 'title'

### Create Dataframe for Plotly

In [ ]:
%%time

# Create visualization "traces" for edges.
def create_edge_traces(graph_object):
    # Declare Edge & Node Locations on x, y axis.
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edges_df = pd.DataFrame({
        'x':edge_x,
        'y':edge_y
    })

    return edges_df

# Create visualization "traces" for nodes.
def create_node_traces(graph_object):
    node_x = []
    node_y = []
    for node in graph_object.nodes():
        x, y = graph_object.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)

    # Declare Node Attributes
    node_text = []
    node_color = []
    for node, attributes in graph_object.nodes(data=True):
        node_text.append(node)
        node_color.append(attributes['type'])

    node_size = []
    for node, adjacencies in enumerate(graph_object.adjacency()):
        node_size.append(len(adjacencies[1]) * 2)

    nodes_df = pd.DataFrame({
        'x':node_x,
        'y':node_y,
        'text':node_text,
        'type':node_color,
        'size':node_size
    })

    return nodes_df

node_traces = create_node_traces(G)
edge_traces = create_edge_traces(G)

node_traces

CPU times: user 3.11 ms, sys: 256 μs, total: 3.36 ms
Wall time: 3.24 ms


,x,y,text,type,size
0,-0.218766,0.032353,Love,experience,32
1,-0.207644,-0.002587,Intimacy,benefit,4
2,0.278087,-0.322516,"Thinking ""What If?""",experience,34
3,0.283791,-0.314188,Innovative Thinking,benefit,2
4,-0.343724,-0.254876,Alienation,experience,32
...,...,...,...,...,...
291,-0.530828,-0.111360,Anne of Green Gables,title,2
292,-0.545280,-0.096300,Antigone,title,2
293,0.054944,-0.207207,Babbitt,title,2
294,0.049059,1.000000,Batrachomyomachia,title,2


### Build Network Visualization

In [87]:
%%time

edge_trace = go.Scatter(
    x=edge_traces['x'], y=edge_traces['y'],
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

fig = px.scatter(
    node_traces, x="x", y="y", color="type",
    size='size', hover_data=['text']
)

fig.add_traces(edge_trace)

fig

CPU times: user 76.2 ms, sys: 5.98 ms, total: 82.2 ms
Wall time: 132 ms


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['Love'],
                                   ['Thinking "What If?"'],
                                   ['Alienation'],
                                   ['Empathy'],
                                   ['Curiosity'],
                                   ['Gratification'],
                                   ['Paranoia'],
                                   ['Self-Irony'],
                                   ['Self-Acceptance'],
                                   ['Courage'],
                                   ['Identification'],
                                   ['Relearning'],
                                   ['Mind Wandering'],
                                   ['Acceptance of Loss'],
                                   ['Tranquility'],
                                   ['Being Wrong'],
                                   ['Generous Love'],
                                   ['Righteousness'],
                                   ['Suspended Judgment'],
                                   ['Awe'],
                                   ['Optimism'],
                                   ['Superiority'],
                                   ['Ready to Grow'],
                                   ['Released Emotion Brake'],
                                   ['Wonder'],
                                   ['Moral Omniscience'],
                                   ['Self-Trust'],
                                   ['Immersion'],
                                   ['Self-Love'],
                                   ['Confusion'],
                                   ['Dissociation'],
                                   ['Distress'],
                                   ['Double Identification'],
                                   ['Eustress'],
                                   ['Existential Nausea'],
                                   ['Togetherness'],
                                   ['Sober Uplift'],
                                   ['Skepticism'],
                                   ['Nimbleness'],
                                   ['Self-Efficacy']], dtype=object),
              'hovertemplate': ('type=experience<br>x=%{x}<br>y' ... '{customdata[0]}<extra></extra>'),
              'legendgroup': 'experience',
              'marker': {'color': '#636efa',
                         'size': array([32, 34, 32, 28, 26, 28, 24, 26, 20, 18, 20, 18, 18, 12, 16, 14, 14, 12,
                                        16, 12, 10, 10,  8,  6,  8, 10,  6,  6,  8,  6,  4,  4,  4,  4,  4,  4,
                                         4,  4,  4,  4]),
                         'sizemode': 'area',
                         'sizeref': np.float64(0.085),
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': 'experience',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([-2.18766329e-01,  2.78087354e-01, -3.43724256e-01, -5.09370942e-01,
                          -2.80392535e-01, -2.68357156e-01,  2.22285765e-01,  4.02286012e-02,
                           1.82583898e-01,  2.58467611e-01,  4.18366242e-01,  1.65315236e-01,
                          -4.27126902e-01,  3.81969563e-01, -1.78735576e-02,  8.42431893e-01,
                          -4.93120726e-01,  4.61172004e-01, -1.26702627e-01,  7.02440435e-01,
                          -7.70910287e-01,  6.01053797e-02,  5.77682583e-01, -3.52228640e-01,
                           7.74995254e-01,  4.04444241e-01, -2.16711730e-01, -3.91357254e-01,
                          -1.75055768e-01, -7.81839335e-01, -7.65925007e-05, -2.93046522e-01,
                           5.03728554e-01,  8.25071613e-01, -2.48826746e-01, -9.16356054e-01,
                          -4.39531179e-01,  1.61753620e-01,  6.35581263e-01, -3.67326844e-01]),
              'xaxis': 'x',
              'y': array([ 0.03235308, -0.32251585, -0.25487618, -0.07438239, -0.79613835,
                   

In [74]:
%%time

# Declare Edge & Node Locations on x, y axis.
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='Rainbow',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title=dict(
              text='Node Connections',
              side='right'
            ),
            xanchor='left',
        ),
        line_width=2))

# Declare Node Attributes
node_text = []
node_color = []
for node, attributes in G.nodes(data=True):
    node_text.append(node)
    node_color.append(attributes['type'])

node_size = []
for node, adjacencies in enumerate(G.adjacency()):
    node_size.append(len(adjacencies[1]) * 2)

# node_trace.marker.color = node_color
node_trace.marker.size = node_size
node_trace.text = node_text

# Visualize
fig = go.Figure(data=[edge_trace, node_trace],
    layout=go.Layout(
        title=dict(
            text="<br>Network graph made with Python",
            font=dict(
                size=16
            )
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20,l=5,r=5,t=40),
        # annotations=[ dict(
        #     text="Python code: <a href='https://plotly.com/python/network-graphs/'> https://plotly.com/python/network-graphs/</a>",
        #     showarrow=False,
        #     xref="paper", yref="paper",
        #     x=0.005, y=-0.002 ) ],
        # xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        # yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
        )
)

fig

ValueError: 
    Invalid element(s) received for the 'color' property of scatter.marker
        Invalid elements include: ['experience', 'benefit', 'experience', 'benefit', 'experience', 'benefit', 'experience', 'benefit', 'experience', 'benefit']

    The 'color' property is a color and may be specified as:
      - A hex string (e.g. '#ff0000')
      - An rgb/rgba string (e.g. 'rgb(255,0,0)')
      - An hsl/hsla string (e.g. 'hsl(0,100%,50%)')
      - An hsv/hsva string (e.g. 'hsv(0,100%,100%)')
      - A named CSS color:
            aliceblue, antiquewhite, aqua, aquamarine, azure,
            beige, bisque, black, blanchedalmond, blue,
            blueviolet, brown, burlywood, cadetblue,
            chartreuse, chocolate, coral, cornflowerblue,
            cornsilk, crimson, cyan, darkblue, darkcyan,
            darkgoldenrod, darkgray, darkgrey, darkgreen,
            darkkhaki, darkmagenta, darkolivegreen, darkorange,
            darkorchid, darkred, darksalmon, darkseagreen,
    

## PyVis Network Visualization

In [ ]:
%%time

nt = Network('500px', '100%')
# nt.from_nx(graph)
nt.generate_html(local=False)

nt

## Tree Map

In [31]:
%%time

tree = px.data.gapminder().query("year == 2007")
tree['continent'].unique()

CPU times: user 4.5 ms, sys: 2.18 ms, total: 6.68 ms
Wall time: 6.69 ms


array(['Asia', 'Europe', 'Africa', 'Americas', 'Oceania'], dtype=object)

In [ ]:
%%time

fig = px.treemap(tree, path=[px.Constant("world"), 'continent', 'country'], values='pop',
                  color='lifeExp', hover_data=['iso_alpha'],
                  color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(tree['lifeExp'], weights=tree['pop']))
                  
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
# fig.show()

CPU times: user 39.9 ms, sys: 1.61 ms, total: 41.5 ms
Wall time: 41.1 ms


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'branchvalues': 'total',
              'customdata': array([['AFG', 43.828],
                                   ['ALB', 76.423],
                                   ['DZA', 72.301],
                                   ...,
                                   ['(?)', 77.89057081069897],
                                   ['(?)', 81.06215400970112],
                                   ['(?)', 68.91909251904043]], dtype=object),
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>pop=%{value' ... 'ifeExp=%{color}<extra></extra>'),
              'ids': array(['world/Asia/Afghanistan', 'world/Europe/Albania',
                            'world/Africa/Algeria', 'world/Africa/Angola',
                            'world/Americas/Argentina', 'world/Oceania/Australia',
                            'world/Europe/Austria', 'world/Asia/Bahrain', 'world/Asia/Bangladesh',
                            'world/Europe/Belgium', 'world/Africa/Benin', 'world/Americas/Bolivia',
                            'world/Europe/Bosnia and Herzegovina', 'world/Africa/Botswana',
                            'world/Americas/Brazil', 'world/Europe/Bulgaria',
                            'world/Africa/Burkina Faso', 'world/Africa/Burundi',
                            'world/Asia/Cambodia', 'world/Africa/Cameroon', 'world/Americas/Canada',
                            'world/Africa/Central African Republic', 'world/Africa/Chad',
                            'world/Americas/Chile', 'world/Asia/China', 'world/Americas/Colombia',
                            'world/Africa/Comoros', 'world/Africa/Congo, Dem. Rep.',
                            'world/Africa/Congo, Rep.', 'world/Americas/Costa Rica',
                            "world/Africa/Cote d'Ivoire", 'world/Europe/Croatia',
                            'world/Americas/Cuba', 'world/Europe/Czech Republic',
                            'world/Europe/Denmark', 'world/Africa/Djibouti',
                            'world/Americas/Dominican Republic', 'world/Americas/Ecuador',
                            'world/Africa/Egypt', 'world/Americas/El Salvador',
                            'world/Africa/Equatorial Guinea', 'world/Africa/Eritrea',
                            'world/Africa/Ethiopia', 'world/Europe/Finland', 'world/Europe/France',
                            'world/Africa/Gabon', 'world/Africa/Gambia', 'world/Europe/Germany',
                            'world/Africa/Ghana', 'world/Europe/Greece', 'world/Americas/Guatemala',
                            'world/Africa/Guinea', 'world/Africa/Guinea-Bissau',
                            'world/Americas/Haiti', 'world/Americas/Honduras',
                            'world/Asia/Hong Kong, China', 'world/Europe/Hungary',
                            'world/Europe/Iceland', 'world/Asia/India', 'world/Asia/Indonesia',
                            'world/Asia/Iran', 'world/Asia/Iraq', 'world/Europe/Ireland',
                            'world/Asia/Israel', 'world/Europe/Italy', 'world/Americas/Jamaica',
                            'world/Asia/Japan', 'world/Asia/Jordan', 'world/Africa/Kenya',
                            'world/Asia/Korea, Dem. Rep.', 'world/Asia/Korea, Rep.',
                            'world/Asia/Kuwait', 'world/Asia/Lebanon', 'world/Africa/Lesotho',
                            'world/Africa/Liberia', 'world/Africa/Libya', 'world/Africa/Madagascar',
                            'world/Africa/Malawi', 'world/Asia/Malaysia', 'world/Africa/Mali',
                            'world/Africa/Mauritania', 'world/Africa/Mauritius',
                            'world/Americas/Mexico', 'world/Asia/Mongolia',
                            'world/Europe/Montenegro', 'world/Africa/Morocco',
                            'world/Africa/Mozambique', 'world/Asia/Myanmar', 'world/Africa/Namibia',
                            'world/Asia/Nepal', 'world/Europe/Netherlands',
                            'world/Ocea

In [37]:
%%time

# groupby : source narrative, experience, benefit, reader

tree = data.groupby(['source_narrative', 'experience']).size().to_frame('count').reset_index()

fig = px.treemap(tree, path=[px.Constant("All"), 'source_narrative', 'experience'], values='count',
                  color='experience') #, hover_data=['iso_alpha'],
                #   color_continuous_s cale='RdBu',
                #   color_continuous_midpoint=np.average(tree['lifeExp'], weights=tree['pop']))
                  
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))

CPU times: user 30.7 ms, sys: 6.19 ms, total: 36.9 ms
Wall time: 35.6 ms


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'branchvalues': 'total',
              'customdata': array([['Courage'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Wonder and Fear'],
                                   ['Wonder and Fear'],
                                   ['Wonder and Fear'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Wonder and Fear'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['Wonder and Fear'],
                                   ['Wonder and Fear'],
                                   ['Courage'],
                                   ['Wonder and Fear'],
                                   ['Courage'],
                                   ['Wonder and Fear'],
                                   ['Courage'],
                                   ['Courage'],
                                   ['(?)']], dtype=object),
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': ('labels=%{label}<br>count=%{val' ... '{customdata[0]}<extra></extra>'),
              'ids': array(['All/A Song of Ice and Fire/Courage',
                            'All/A Tale of Two Cities/Courage', 'All/Beloved/Courage',
                            'All/The Iliad/Courage', 'All/Book of the Dead/Wonder and Fear',
                            'All/Pyramid Texts/Wonder and Fear',
                            'All/Sanskrit Rigveda/Wonder and Fear', 'All/Peyton Place/Courage',
                            "All/Ridley Scott's Gladiator/Courage", 'All/The Illiad/Courage',
                            'All/Genesis/Wonder and Fear', 'All/A Song of Ice and Fire',
                            'All/A Tale of Two Cities', 'All/Beloved', 'All/Book of the Dead',
                            'All/Genesis', 'All/Peyton Place', 'All/Pyramid Texts',
                            "All/Ridley Scott's Gladiator", 'All/Sanskrit Rigveda', 'All/The Iliad',
                            'All/The Illiad', 'All'], dtype=object),
              'labels': array(['Courage', 'Courage', 'Courage', 'Courage', 'Wonder and Fear',
                               'Wonder and Fear', 'Wonder and Fear', 'Courage', 'Courage', 'Courage',
                               'Wonder and Fear', 'A Song of Ice and Fire', 'A Tale of Two Cities',
                               'Beloved', 'Book of the Dead', 'Genesis', 'Peyton Place',
                               'Pyramid Texts', "Ridley Scott's Gladiator", 'Sanskrit Rigveda',
                               'The Iliad', 'The Illiad', 'All'], dtype=object),
              'marker': {'colors': [#636efa, #636efa, #636efa, #636efa, #EF553B,
                                    #EF553B, #EF553B, #636efa, #636efa, #636efa,
                                    #EF553B, #636efa, #636efa, #636efa, #EF553B,
                                    #EF553B, #636efa, #EF553B, #636efa, #EF553B,
                                    #636efa, #636efa, #00cc96]},
              'name': '',
              'parents': array(['All/A Song of Ice and Fire', 'All/A Tale of Two Cities', 'All/Beloved',
                                'All/The Iliad', 'All/Book of the Dead', 'All/Pyramid Texts',
                                'All/Sanskrit Rigveda', 'All/Peyton Place',
                                "All/Ridley Scott's Gladiator", 'All/The Illiad', 'All/Genesis', 'All',
                                'All', 'All', 'All', 'All', 'All', 'All', 'All', 'All', 'All', 'All',
                                ''], dtype=object),
              'type': 'treemap',
              'values': array([ 2,  2,  2,  1,  1,  2,  2,  2, 70,  1,  1,  2,  2,  2,  1,  1,  2,  2,
                       

## Packed Circles

In [11]:
%%time

# data
# compute circle positions
circles = circlify.circlify(
    data['source_narrative'].tolist(),
    show_enclosure=False,
    target_enclosure=circlify.Circle(x=0, y=0, r=1)
)

# reverse the order of the circles to match the order of data
circles = circles[::-1]

TypeError: '<=' not supported between instances of 'str' and 'float'

## Timeline

In [44]:
# df[pandas.notnull(df['mean'])]

data[pd.notnull(data['pubDate'])]['pubDate']

1    1894-01-01 00:00:00+00:00
2    1894-01-01 00:00:00+00:00
8    1987-09-01 00:00:00+00:00
9    1987-09-01 00:00:00+00:00
10   1859-01-01 00:00:00+00:00
                ...           
81   2000-05-05 00:00:00+00:00
82   2023-05-19 00:00:00+00:00
83   2000-05-19 00:00:00+00:00
84   1996-01-01 00:00:00+00:00
85   1996-01-01 00:00:00+00:00
Name: pubDate, Length: 78, dtype: datetime64[ns, UTC]

In [47]:
%%time

timeline_data = data[pd.notnull(data['pubDate'])]
# fig = px.timeline(data, x_start="pubDate", x_end="pubDate", y="occurrence_date",color="occurrence_type")

fig = px.scatter(timeline_data, x = 'pubDate', y = 'benefit', color='benefit')

# Add range slider
fig.update_layout(
    xaxis=dict(
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

CPU times: user 25.9 ms, sys: 4.62 ms, total: 30.5 ms
Wall time: 60.9 ms


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'benefit=%{y}<br>pubDate=%{x}<extra></extra>',
              'legendgroup': 'Obedience',
              'marker': {'color': '#636efa', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Obedience',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(1894, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(1894, 1, 1, 0, 0, tzinfo=datetime.timezone.utc)],
                         dtype=object),
              'xaxis': 'x',
              'y': array(['Obedience', 'Obedience'], dtype=object),
              'yaxis': 'y'},
             {'hovertemplate': 'benefit=%{y}<br>pubDate=%{x}<extra></extra>',
              'legendgroup': 'Tend and Befriend',
              'marker': {'color': '#EF553B', 'symbol': 'circle'},
              'mode': 'markers',
              'name': 'Tend and Befriend',
              'orientation': 'v',
              'showlegend': True,
              'type': 'scatter',
              'x': array([datetime.datetime(1987, 9, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(1987, 9, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(1859, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(1859, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 5, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 25, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2023, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2000, 5, 19, 0, 0, tzinfo=datetime.timezone.utc),
             